In [24]:
import numpy as np

import random
from collections import deque
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Normal

import torch.optim as optim

import cv2

import game

from IPython.display import clear_output

import random
from collections import deque

In [25]:
from importlib import reload
reload(game)

<module 'game' from '/m/home/home7/70/viitala1/data/Documents/Dippa/driving_simulator/game.py'>

In [26]:
import pygame

In [27]:
pygame.display.quit()

In [28]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

class Conv(nn.Module):
    def __init__(self, img_channels):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(img_channels, 16, 5, stride=2, padding=2),
            nn.ReLU(),
            nn.Conv2d(16, 16, 5, stride=2, padding=2),
            nn.ReLU(),
            nn.Conv2d(16, 16, 5, stride=2, padding=2),
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, stride=1, padding=1),
            nn.ReLU(),
            Flatten(),
        )
    def forward(self, x):
        #print(x.shape)
        #inp = x.reshape((64, 1, 80, 80))
        #inp = x.unsqueeze(0)
        return self.net(x)

In [29]:
class Critic(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Critic, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.linear3 = nn.Linear(hidden_size, output_size)

    def forward(self, state, action):
        """
        Params state and actions are torch tensors
        """
        x = torch.cat([state, action], 1)
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)

        return x

In [30]:
class Actor(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, learning_rate = 3e-4):
        super(Actor, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.linear3 = nn.Linear(hidden_size, output_size)
        
    def forward(self, state):
        """
        Param state is a torch tensor
        """
        x = F.relu(self.linear1(state))
        x = F.relu(self.linear2(x))
        x = torch.tanh(self.linear3(x))

        return x

In [31]:
class Memory:
    def __init__(self, max_size):
        self.buffer = deque(maxlen=max_size)
    
    def push(self, state, action, reward, next_state, done):
        experience = (state, action, np.array([reward]), next_state, done)
        self.buffer.append(experience)

    def sample(self, batch_size):
        state_batch = []
        action_batch = []
        reward_batch = []
        next_state_batch = []
        done_batch = []

        batch = random.sample(self.buffer, batch_size)

        for experience in batch:
            state, action, reward, next_state, done = experience
            state_batch.append(state)
            action_batch.append(action)
            reward_batch.append(reward)
            next_state_batch.append(next_state)
            done_batch.append(done)
        
        return state_batch, action_batch, reward_batch, next_state_batch, done_batch

    def __len__(self):
        return len(self.buffer)

In [32]:
class OUNoise():
    def __init__(self, mu=0.0, theta=0.1, max_sigma=0.01, min_sigma=0.01, decay_period=100000):
        self.mu           = mu
        self.theta        = theta
        self.sigma        = max_sigma
        self.max_sigma    = max_sigma
        self.min_sigma    = min_sigma
        self.decay_period = decay_period
        self.action_dim   = 2
        self.low          = -0.1
        self.high         = 0.1
        self.reset()
        
    def reset(self):
        self.state = np.ones(self.action_dim) * self.mu
        
    def evolve_state(self):
        x  = self.state
        dx = self.theta * (self.mu - x) + self.sigma * np.random.randn(self.action_dim)
        self.state = x + dx
        return self.state
    
    def get_action(self, action, t=0): 
        ou_state = self.evolve_state()
        self.sigma = self.max_sigma - (self.max_sigma - self.min_sigma) * min(1.0, t / self.decay_period)
        return np.clip(action + ou_state, self.low, self.high)

In [35]:
class DDPG_agent:
    
    def __init__(self, actor_learning_rate=1e-4, critic_learning_rate=1e-4, batch_size=64, tau=0.001, gamma=0.0001, ):
        
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        self.batch_size = batch_size
        
        self.gamma = gamma
        self.tau = tau
        
        self.actor = Actor(1600, 100, 2).to(device)
        self.actor_target = Actor(1600, 100, 2).to(device)
        
        self.critic = Critic(1602, 100, 2).to(device)
        self.critic_target = Critic(1602, 100, 2).to(device)
        
        self.conv = Conv(4).to(device)
        
        self.memory = Memory(10000)
        
        for target_param, param in zip(self.actor_target.parameters(), self.actor.parameters()):
            target_param.data.copy_(param.data)

        for target_param, param in zip(self.critic_target.parameters(), self.critic.parameters()):
            target_param.data.copy_(param.data)
            
        self.critic_criterion  = nn.MSELoss()
        self.actor_optimizer  = optim.Adam(self.actor.parameters(), lr=actor_learning_rate)
        self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=critic_learning_rate)
        
    def get_action(self, state):
        embed = self.conv.forward(state)
        action = self.actor.forward(embed).detach().cpu().numpy()
        return action
        
    def update(self):
        
        states, actions, rewards, next_states, _ = self.memory.sample(self.batch_size)
        states = torch.FloatTensor(states).to(device)
        actions = torch.FloatTensor(actions).to(device)
        rewards = torch.FloatTensor(rewards).to(device)
        
        #print(np.array(next_states).shape)
        next_states = torch.FloatTensor(next_states).to(device)
        
        states = self.conv.forward(states).to(device)
        next_states = self.conv.forward(next_states)
        
        #print(states.shape)
        #print(actions.shape)
        
        Qvals = self.critic.forward(states, actions)
        next_actions = self.actor_target.forward(next_states)
        next_Q = self.critic_target.forward(next_states, next_actions.detach())
        Qprime = rewards + self.gamma * next_Q
        
        critic_loss = self.critic_criterion(Qvals, Qprime)
        
        policy_loss = -self.critic.forward(states, self.actor.forward(states)).mean()
        
        # update networks
        self.actor_optimizer.zero_grad()
        policy_loss.backward(retain_graph=True)
        self.actor_optimizer.step()

        self.critic_optimizer.zero_grad()
        critic_loss.backward() 
        self.critic_optimizer.step()

        # update target networks 
        for target_param, param in zip(self.actor_target.parameters(), self.actor.parameters()):
            target_param.data.copy_(param.data * self.tau + target_param.data * (1.0 - self.tau))
       
        for target_param, param in zip(self.critic_target.parameters(), self.critic.parameters()):
            target_param.data.copy_(param.data * self.tau + target_param.data * (1.0 - self.tau))

In [37]:
env = game.game()

agent = DDPG_agent()
noise = OUNoise()
batch_size = 64
rewards = []
avg_rewards = []

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

counter = 0

exploration = 10000



for episode in range(50):
    state = env.reset()
    
    obs = state
    noise.reset()
    episode_reward = 0
    
    state = np.stack((state, state, state, state), axis = 0)[np.newaxis, :]
    
    done= False
    while not done:
        action = agent.get_action(torch.FloatTensor(state).to(device)) / 10
        clear_output(wait = True)
        print(action)
        
        scale = (exploration - counter) / exploration
        action = action + np.random.normal(scale = scale, size = 2)
        print(action)
        #action = noise.get_action(action, counter)
        #print(action[0, 0])
        obs, reward, done = env.step(action[0, 0], 1, obs)
        
        new_state = obs
        
        #print(state.shape)
        new_state = new_state[np.newaxis, np.newaxis, :, :]
        new_state = np.append(new_state, state[:,:3,:,:], axis=1)
        
        #print(np.array(new_state.shape))
        
        agent.memory.push(state.squeeze(), action.squeeze(), reward, new_state.squeeze(), done)
        
        
        if len(agent.memory) > batch_size:
            agent.update()        
        
        
        counter += 1
        
        state = new_state
        episode_reward += reward

        if done:
            print("episode: {}, reward: {}, average _reward: {} \n".format(episode, np.round(episode_reward, decimals=2), np.mean(rewards[-10:])))
            break


[[ 0.1 -0.1]]
[[-0.00383249 -1.46643683]]
episode: 49, reward: 153.46, average _reward: nan 



In [ ]:
x